In [1]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split, Subset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch.nn.functional as F
import matplotlib.pyplot as plt

is_qwen=True
ce_data_dir = "../cbm/coco_caption/"
prefix = "origin"
input_datas = torch.load(ce_data_dir+f'{prefix}coco_caption_ce_val_full_v1.pth')
input_datas_image = torch.load(ce_data_dir+f'{prefix}coco_caption_ce_val_image_full_v1.pth')


In [2]:
missing_index = [73, 98, 188, 190, 232, 235, 258, 267, 275, 287, 297, 309, 316, 338, 365, 423, 470, 480, 483, 657, 1256, 1284, 1323, 1453, 1484, 1654, 1701, 1717, 1726, 1729, 1794, 1796, 1815, 1830, 1871, 1891, 1942, 1954, 2004, 2017, 2037, 2038, 2082, 2096, 2144, 2207, 2282, 2373, 2477, 2502, 2529, 2631, 2778, 2850, 2912, 2941, 2952, 2953, 3013, 3022, 3049, 3067, 3069, 3081, 3185, 3188, 3196, 3202, 3212, 3242, 3296, 3309, 3344, 3355, 3494, 3611, 3687, 3699, 3797, 3824, 3887, 3892, 3999, 4003, 4081, 4819, 4855, 4862, 4980, 5001, 5029, 5096, 5112, 5189, 5280, 5309, 5393, 5523, 5537, 5618, 5669, 5784, 5815, 5854, 5992, 6059, 6063, 6101, 6125, 6312, 6456, 6590, 6597, 6611, 6632, 6666, 6720, 6728, 6731, 6763, 7493, 7523, 7605, 7675, 7694, 7717, 7773, 7778, 7892, 7895, 8206, 8521, 8533, 8689, 8707, 8759, 8802, 8821, 8825, 8921, 8951, 9020, 9021, 9073, 9112, 9134, 9139, 9147, 9251, 9257, 9320, 9363, 9430, 9431, 9440, 9477, 9484, 9490, 9518, 9538, 9541, 9564, 9603, 9606, 9689, 9721, 9740, 9754, 9864, 9932, 9974, 9992]
for i in range(len(input_datas)):
    input_datas[i]=[x for i, x in enumerate(input_datas[i]) if i not in missing_index]
    input_datas_image[i]=[x for i, x in enumerate(input_datas_image[i]) if i not in missing_index]


In [3]:
is_qwen=True
all_kls = []

if is_qwen:
    info_save_index=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36]
else:
    info_save_index=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32]


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt



# 初始化结果容器
all_metrics = {
    # 'kl': [],
    # 'cosine': [],
    # 'euclidean': [],
    'js': [],
    # 'pearson': []
}

# # 添加平滑常数和极小值
# EPSILON = 1e-15
# MIN_VALUE = 1e-8

# for i, index in enumerate(info_save_index):
#     print(f"-----------------index {index}-----------------")
    
#     # 加载数据
#     input_data = input_datas[i]
#     input_data_image = input_datas_image[i]
    
#     data_tensor = torch.stack(input_data).float()
#     data_tensor_image = torch.stack(input_data_image).float()
    
#     # 创建数据集和加载器
#     dataset = TensorDataset(data_tensor, data_tensor_image)
#     loader = DataLoader(dataset, batch_size=128, shuffle=False)
    
#     # 初始化指标
#     total_js = 0.0
#     total_samples = 0
    
#     for text_batch, image_batch in loader:
#         batch_size = text_batch.size(0)
        
#         # 计算概率分布（添加平滑处理）
#         prob_text = F.softmax(text_batch, dim=1) + EPSILON
#         prob_image = F.softmax(image_batch, dim=1) + EPSILON
        
#         # 重新归一化确保有效概率分布
#         prob_text = prob_text / prob_text.sum(dim=1, keepdim=True)
#         prob_image = prob_image / prob_image.sum(dim=1, keepdim=True)
        
#         # 计算中间分布M（添加平滑处理）
#         M = 0.5 * (prob_text + prob_image) + EPSILON
#         M = M / M.sum(dim=1, keepdim=True)  # 重新归一化
        
#         # 安全计算对数概率（避免log(0)）
#         log_prob_text = torch.log(torch.clamp(prob_text, min=MIN_VALUE))
#         log_prob_image = torch.log(torch.clamp(prob_image, min=MIN_VALUE))
#         log_M = torch.log(torch.clamp(M, min=MIN_VALUE))
        
#         # 计算KL散度（使用稳定版本）
#         kl_text = F.kl_div(log_prob_text, M, reduction='none', log_target=False).sum(dim=1)
#         kl_image = F.kl_div(log_prob_image, M, reduction='none', log_target=False).sum(dim=1)
        
#         # 计算JS散度
#         js_batch = 0.5 * kl_text + 0.5 * kl_image
        
#         # 检测并处理无效值
#         invalid_mask = torch.isnan(js_batch) | torch.isinf(js_batch)
#         if invalid_mask.any():
#             print(f"Warning: {invalid_mask.sum().item()}/{batch_size} invalid JS values detected")
            
#             # 获取有效值的均值作为替代值
#             valid_js = js_batch[~invalid_mask]
#             replacement = valid_js.mean() if len(valid_js) > 0 else 0.0
#             js_batch = torch.where(invalid_mask, torch.tensor(replacement, device=js_batch.device), js_batch)
        
#         total_js += js_batch.sum().item()
#         total_samples += batch_size
    
#     # 计算平均JS值
#     avg_js = total_js / total_samples if total_samples > 0 else 0.0
#     all_metrics['js'].append(avg_js)


for i, index in enumerate(info_save_index):
    print(f"-----------------index {index}-----------------")
    
    # 加载数据
    input_data = input_datas[i]
    input_data_image = input_datas_image[i]
    
    data_tensor = torch.stack(input_data).float()
    data_tensor_image = torch.stack(input_data_image).float()
    
    # 创建数据集和加载器
    dataset = TensorDataset(data_tensor, data_tensor_image)
    loader = DataLoader(dataset, batch_size=128, shuffle=False)
    
    # 初始化指标
    total_kl = 0.0
    total_cosine = 0.0
    total_euclidean = 0.0
    total_js = 0.0
    total_pearson = 0.0
    total_samples = 0
    
    # if i<32:
    #     continue
    
    for text_batch, image_batch in loader:
        batch_size = text_batch.size(0)

        # # # 1. KL散度
        # log_text = F.log_softmax(text_batch, dim=1)
        prob_image = F.softmax(image_batch, dim=1)
        
        # print("text_batch", text_batch.shape)
        # print("prob_image", prob_image.shape)
        
        # kl_batch = F.kl_div(log_text, prob_image, reduction='none').sum(dim=1)
        # total_kl += kl_batch.sum().item()
        
        # # 2. 余弦相似度
        # cosine_batch = F.cosine_similarity(text_batch, image_batch, dim=1)
        # total_cosine += cosine_batch.sum().item()
        
        # # # # 3. 欧氏距离
        # euclidean_batch = torch.norm(text_batch - image_batch, p=2, dim=1)
        # # print(euclidean_batch)
        
        # total_euclidean += euclidean_batch.sum().item()
        
        # # 4. JS散度
        prob_text = F.softmax(text_batch, dim=1)
        # M = 0.5 * (prob_text + prob_image)
        
        # if M.sum()<EPSILON:
        #     M=EPSILON
        
        # if torch.isnan(prob_text).any():
        #     print(f"Tensor {i} contains NaN.")
        # if torch.isinf(prob_text).any():
        #     print(f"Tensor {i} contains Inf.")
        
        # if torch.isnan(prob_image).any():
        #     print(f"Tensor {i} contains NaN.")
        # if torch.isinf(prob_image).any():
        #     print(f"Tensor {i} contains Inf.")
        js_batch=[]
        for i in range(prob_text.shape[0]):
            # 取当前样本的概率分布
            p_text = prob_text[i]
            p_image = prob_image[i]
            
                
            # 计算平均分布
            p_avg = 0.5 * (p_text + p_image)
            
            # 计算 KL 散度
            kl_div_text = F.kl_div(torch.log(p_text), p_avg, reduction='batchmean')
            kl_div_image = F.kl_div(torch.log(p_image), p_avg, reduction='batchmean')
            # kl_div_text = torch.nn.functional.kl_div(p_text.log(), p_avg, reduction='batchmean')
            # kl_div_image = torch.nn.functional.kl_div(p_image.log(), p_avg, reduction='batchmean')
            
            # if torch.isinf(kl_div_text) or torch.isnan(kl_div_text):
            #     kl_div_text = torch.tensor(torch.log(2))
            #     # print("other part kl_div_image", kl_div_image)
            #     # if torch.isinf(kl_div_image) or torch.isnan(kl_div_image):
            #     #     print("critical", p_text, p_image)
            #     #     print(max(p_text), min(p_text))
            #     #     print(max(p_image), min(p_image))
                    
            # if torch.isinf(kl_div_image) or torch.isnan(kl_div_image):
            #     kl_div_image = torch.tensor(torch.log(2))
            #     # print("other part kl_div_text", kl_div_text)
            
            # if (torch.isinf(kl_div_text) or torch.isnan(kl_div_text)) | (torch.isinf(kl_div_image) or torch.isnan(kl_div_image)) == True:
            #     print("!!!!!!!!!!!")
            
            # if (torch.isinf(kl_div_text) or torch.isnan(kl_div_text)) and (torch.isinf(kl_div_image) or torch.isnan(kl_div_image)):
            #     # print("avg", p_avg)
            #     kl_div_text = kl_div_image = torch.log(2)
            
            if (torch.isinf(kl_div_text) or torch.isnan(kl_div_text)) or (torch.isinf(kl_div_image) or torch.isnan(kl_div_image)):
                kl_div_text = kl_div_image = torch.log(torch.tensor(2))/p_text.shape[0]
            
            # 计算 JS 散度
            js_div = torch.tensor(0.5 * kl_div_text + 0.5 * kl_div_image)
            
            if torch.isinf(js_div) or torch.isnan(js_div):
                print("avg", p_avg)
                js_div = torch.tensor(torch.log(torch.tensor(2))/p_text.shape[0])
                print("TENSOR1", p_text, "TENSOR2", p_image)
                print(kl_div_text, kl_div_image)
                print("max", max(p_text), max(p_image))
                print("min", min(p_text), min(p_image))
                print("norm", torch.norm(text_batch - image_batch, p=2))
        
            # print(kl_div_text, kl_div_image)
            js_batch.append(js_div)

        # 将结果转换为张量
        js_batch = torch.tensor(js_batch)
        # print(len(js_batch), js_batch[0].shape)
        # js_batch = 0.5 * (F.kl_div(torch.log(prob_text), M, reduction='none').sum(dim=1)) + \
        #            0.5 * (F.kl_div(torch.log(prob_image), M, reduction='none').sum(dim=1))
        
        # print(js_batch)
        
        # def check_nan_inf_and_print(tensor, name):
        #     if torch.isnan(tensor).any():
        #         print(f"{name} contains NaN.")
        #         print(f"Tensor {name} stats: min={tensor.min()}, max={tensor.max()}, mean={tensor.mean()}")
        #     if torch.isinf(tensor).any():
        #         print(f"{name} contains Inf.")
        #         print(f"Tensor {name} stats: min={tensor.min()}, max={tensor.max()}, mean={tensor.mean()}")
        
        # # 计算 KL 散度
        # kl_text = F.kl_div(torch.log(prob_text), M, reduction='none')
        # kl_image = F.kl_div(torch.log(prob_image), M, reduction='none')

        # # 检查 KL 散度
        # check_nan_inf_and_print(kl_text, "kl_text")
        # check_nan_inf_and_print(kl_image, "kl_image")

        # valid_js = js_batch[~torch.isnan(js_batch) & ~torch.isinf(js_batch)]
        # print(js_batch)
        total_js += sum(js_batch) #.item()
        
        # print(js_batch)
        # print("max", max(js_batch))
        # # # 5. 皮尔逊相关系数
        # mean_text = text_batch.mean(dim=1, keepdim=True)
        # mean_image = image_batch.mean(dim=1, keepdim=True)
        # cov = ((text_batch - mean_text) * (image_batch - mean_image)).sum(dim=1)
        # std_text = torch.sqrt(((text_batch - mean_text)**2).sum(dim=1))
        # std_image = torch.sqrt(((image_batch - mean_image)**2).sum(dim=1))
        # pearson_batch = cov / (std_text * std_image + 1e-8)  # 避免除零
        # total_pearson += pearson_batch.sum().item()
        
        total_samples += batch_size
    
    # 计算平均指标
    # all_metrics['kl'].append(total_kl / total_samples)
    # all_metrics['cosine'].append(total_cosine / total_samples)
    # all_metrics['euclidean'].append(total_euclidean / total_samples)
    print("append", total_js / total_samples)
    all_metrics['js'].append(total_js / total_samples)
    # all_metrics['pearson'].append(total_pearson / total_samples)



-----------------index 0-----------------


/tmp/ipykernel_1625783/3401418059.py:187: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  js_div = torch.tensor(0.5 * kl_div_text + 0.5 * kl_div_image)


append tensor(6.6022e-08)
-----------------index 1-----------------
append tensor(0.0001)
-----------------index 2-----------------
append tensor(0.0002)
-----------------index 3-----------------
append tensor(0.0002)
-----------------index 4-----------------
append tensor(0.0003)
-----------------index 5-----------------
append tensor(0.0006)
-----------------index 6-----------------
append tensor(0.0012)
-----------------index 7-----------------
append tensor(0.0016)
-----------------index 8-----------------
append tensor(0.0016)
-----------------index 9-----------------
append tensor(0.0016)
-----------------index 10-----------------
append tensor(0.0013)
-----------------index 11-----------------
append tensor(0.0006)
-----------------index 12-----------------
append tensor(0.0004)
-----------------index 13-----------------
append tensor(0.0004)
-----------------index 14-----------------
append tensor(0.0010)
-----------------index 15-----------------
append tensor(0.0012)
--------

In [13]:
# 保存结果
for metric_name, values in all_metrics.items():
    with open(f'result/{prefix}{metric_name}_layerwise.txt', 'w') as f:
        for value in values:
            f.write(f"{value}\n")
    
    # 绘制图表
    plt.figure(figsize=(10, 6))
    plt.plot(values, marker='o')
    plt.title(f'Layer-wise {metric_name.upper()} Comparison')
    plt.xlabel('Layer Index')
    plt.ylabel(metric_name.capitalize())
    plt.grid(True)
    plt.savefig(f'result/{prefix}{metric_name}_layerwise.pdf', format='pdf')
    plt.close()

# # 保存结果到文件
# written_file = f'result/{prefix}direct_kls_layerwise.txt'
# with open(written_file, 'w') as f:
#     for kl in all_kls:
#         f.write(f"{kl}\n")
# print(f"Results written to: {written_file}")

# # 绘制折线图
# plt.figure(figsize=(10, 6))
# plt.plot(all_kls[:-1], marker='o', linestyle='-', color='b')
# plt.title('Direct KL Divergence between Text and Image Features')
# plt.xlabel('Layer Index')
# plt.ylabel('KL Divergence')
# plt.grid(True)
# plt.savefig(f'result/{prefix}direct_kls_layerwise.pdf', format='pdf')
# plt.close()
# print("Plot saved as PDF")